# Find potential conflicts between german db and TPA1

In [3]:
import pandas as pd
from IPython.display import display
from tqdm.notebook import tqdm
from ITUtils import itu_to_bandwidth, channel_appender
from pandarallel import pandarallel
import numpy as np

# Initialize pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
# TPA reference table
tpatablename = './databases/TPAtable.csv'
# germany ITU database (obtained filtering by group frequency ranges)
germanyname = './databases/germany.csv'

# import as pandas pd
tpa = pd.read_csv(tpatablename)
ddf = pd.read_csv(germanyname)

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

display(ddf.head())

/tmp/ipykernel_159606/138441510.py:8: DtypeWarning: Columns (3,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ddf = pd.read_csv(germanyname)


,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,orbit.apog_km,orbit.perig_km,orbit.op_ht_km,s_beam.emi_rcp,s_beam.beam_name,grp.grp_id,grp.freq_min,grp.freq_max,grp.bdwdth,grp.d_rcv,grp.d_inuse,grp.d_reg_limit,grp.d_prot_eff,grp.f_biu,emiss.seq_no,emiss.pwr_ds_max,emiss.design_emi,carrier_fr.freq_carr
0,100590924,NaN,D,NaN,DFS-1,RS49,03.11.2000,50,NaN,NaN,NaN,NaN,NaN,E,RS49,100003997,2201.35,2202.85,NaN,03.11.2000,06.06.1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100590925,NaN,D,NaN,DFS-2,RS49,03.11.2000,50,NaN,NaN,NaN,NaN,NaN,E,RS49,100004004,2201.35,2202.85,NaN,03.11.2000,24.09.1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114500155,NaN,D,EUM,MTG-D1_3.4W,11.2,09.10.2023,50,NaN,NaN,NaN,NaN,NaN,R,UPR,116612200,401.00,403.00,2000.0,25.01.2016,02.08.2015,19.12.2015,23.03.2010,C,3.0,-30.0,3K00G7D--,NaN
3,109500171,NaN,D,NaN,GENESIS-8,11.2,12.06.2012,50,NaN,NaN,NaN,NaN,NaN,E,TTCD,109609736,2212.90,2288.50,5000.0,NaN,02.01.2009,09.01.2010,24.10.2003,C,2.0,-41.9,1M20G9D--,NaN
4,109500171,NaN,D,NaN,GENESIS-8,11.2,12.06.2012,50,NaN,NaN,NaN,NaN,NaN,E,TTCD,109609736,2212.90,2288.50,5000.0,NaN,02.01.2009,09.01.2010,24.10.2003,C,1.0,-41.9,5M00G9D--,NaN


First 40 rows

In [5]:
# add channel info to germany dataframe
ddf = channel_appender(ddf)
print(ddf.info())
display(ddf[:40])
print(ddf.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772078 entries, 0 to 772077
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   com_el.ntc_id          772078 non-null  int64  
 1    com_el.tgt_ntc_id     0 non-null       float64
 2    com_el.adm            772078 non-null  object 
 3    com_el.ntwk_org       70 non-null      object 
 4    com_el.sat_name       772078 non-null  object 
 5    com_el.prov           772078 non-null  object 
 6    com_el.d_rcv          772078 non-null  object 
 7    com_el.st_cur         772078 non-null  int64  
 8    orbit.orb_id          768854 non-null  float64
 9    orbit.nbr_sat_pl      768854 non-null  float64
 10   orbit.apog_km         768854 non-null  float64
 11   orbit.perig_km        768854 non-null  float64
 12   orbit.op_ht_km        768854 non-null  float64
 13   s_beam.emi_rcp        772078 non-null  object 
 14   s_beam.beam_name      772078 non-nu

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,orbit.apog_km,orbit.perig_km,orbit.op_ht_km,s_beam.emi_rcp,s_beam.beam_name,grp.grp_id,grp.freq_min,grp.freq_max,grp.bdwdth,grp.d_rcv,grp.d_inuse,grp.d_reg_limit,grp.d_prot_eff,grp.f_biu,emiss.seq_no,emiss.pwr_ds_max,emiss.design_emi,carrier_fr.freq_carr,channel.bandwidth,channel.freq_min,channel.freq_max
0,100590924,NaN,D,NaN,DFS-1,RS49,03.11.2000,50,NaN,NaN,NaN,NaN,NaN,E,RS49,100003997,2201.3500,2202.850,NaN,03.11.2000,06.06.1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100590925,NaN,D,NaN,DFS-2,RS49,03.11.2000,50,NaN,NaN,NaN,NaN,NaN,E,RS49,100004004,2201.3500,2202.850,NaN,03.11.2000,24.09.1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114500155,NaN,D,EUM,MTG-D1_3.4W,11.2,09.10.2023,50,NaN,NaN,NaN,NaN,NaN,R,UPR,116612200,401.0000,403.000,2000.0,25.01.2016,02.08.2015,19.12.2015,23.03.2010,C,3.0,-30.0,3K00G7D--,NaN,3000.0,NaN,NaN
3,109500171,NaN,D,NaN,GENESIS-8,11.2,12.06.2012,50,NaN,NaN,NaN,NaN,NaN,E,TTCD,109609736,2212.9000,2288.500,5000.0,NaN,02.01.2009,09.01.2010,24.10.2003,C,2.0,-41.9,1M20G9D--,NaN,1200000.0,NaN,NaN
4,109500171,NaN,D,NaN,GENESIS-8,11.2,12.06.2012,50,NaN,NaN,NaN,NaN,NaN,E,TTCD,109609736,2212.9000,2288.500,5000.0,NaN,02.01.2009,09.01.2010,24.10.2003,C,1.0,-41.9,5M00G9D--,NaN,5000000.0,NaN,NaN
5,109500420,NaN,D,NaN,GENESIS-9,11.2,13.08.2012,50,NaN,NaN,NaN,NaN,NaN,R,TTCU,112732217,2037.5000,2107.500,5000.0,13.08.2012,12.10.2009,09.01.2010,24.10.2003,C,1.0,-34.8,200KG9D--,NaN,200000.0,NaN,NaN
6,116545143,NaN,D,NaN,SARAH,9.1/IA,07.07.2020,50,1.0,2.0,500.0,500.0,500.0,R,TCP,116680574,2025.0000,2110.000,NaN,07.07.2020,NaN,26.04.2023,NaN,NaN,1.0,-40.0,90K0G1DCN,2098.4500,90000.0,2098.4050,2098.4950
7,109500420,NaN,D,NaN,GENESIS-9,11.2,13.08.2012,50,NaN,NaN,NaN,NaN,NaN,E,TTCD,109637897,2212.9000,2288.500,5000.0,04.09.2009,12.10.2009,09.01.2010,24.10.2003,C,1.0,-41.9,5M00G9D--,NaN,5000000.0,NaN,NaN
8,111500246,NaN,D,NaN,GENESIS-11,11.2,08.12.2011,50,NaN,NaN,NaN,NaN,NaN,R,TTCU,111703514,2037.5000,2107.500,5000.0,08.12.2011,28.05.2010,16.07.2014,16.01.2008,C,1.0,-11.5,200KG9D--,NaN,200000.0,NaN,NaN
9,114500155,NaN,D,EUM,MTG-D1_3.4W,11.2,09.10.2023,50,NaN,NaN,NaN,NaN,NaN,R,SOR,116612199,2065.3896,2075.598,1000.0,09.10.2023,02.08.2015,19.12.2015,23.03.2010,C,2.0,-20.0,1M00GXX--,NaN,1000000.0,NaN,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.prov', ' com_el.d_rcv',
       ' com_el.st_cur', ' orbit.orb_id', ' orbit.nbr_sat_pl',
       ' orbit.apog_km', ' orbit.perig_km', ' orbit.op_ht_km',
       ' s_beam.emi_rcp', ' s_beam.beam_name', ' grp.grp_id', ' grp.freq_min',
       ' grp.freq_max', ' grp.bdwdth', ' grp.d_rcv', ' grp.d_inuse',
       ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu', ' emiss.seq_no',
       ' emiss.pwr_ds_max', ' emiss.design_emi', ' carrier_fr.freq_carr',
       ' channel.bandwidth', ' channel.freq_min', ' channel.freq_max'],
      dtype='object')


##  Comparison with the TPA datafields


In [6]:
# truncate the database for test
# ddf = ddf[:1000]

In [7]:
# Ensure the tpaconflicts column exists in ddf
if 'tpaconflicts' not in ddf.columns:
    ddf['tpaconflicts'] = ''

# Remove rows with NaN values in 'channel.freq_max' or 'channel.freq_min'
ddf = ddf.dropna(subset=[' channel.freq_max', ' channel.freq_min'])

# Define a function to check for overlaps and update the tpaconflicts column
def check_overlap(ddf_row, tpa):
    conflicts = ''
    for tpa_index, tpa_row in tpa.iterrows():
        tpa_fmax = tpa_row[' channel.freq_max']
        tpa_fmin = tpa_row[' channel.freq_min']
        ddf_fmax = ddf_row[' channel.freq_max']
        ddf_fmin = ddf_row[' channel.freq_min']
        
        # # Skip if any of the ddf_fmin or ddf_fmax is NaN does not work in parallel
        # if np.isnan(float(ddf_fmin)) or np.isnan(ddf_fmax):
        #     continue

        # Check if intervals overlap
        if not (tpa_fmax < ddf_fmin or tpa_fmin > ddf_fmax):
            conflicts += f"{tpa_index}:"
    
    return conflicts

# Apply the function to each row of the ddf DataFrame (takes at least 2 minutes to process all)
tqdm.pandas()
ddf['tpaconflicts'] = ddf.parallel_apply(check_overlap, axis=1, args=(tpa,))

# Filter out non-conflicting rows
ddf_conflicts = ddf[ddf['tpaconflicts'] != '']

# Display the updated ddf DataFrame with conflicts
display(ddf_conflicts)

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,orbit.apog_km,orbit.perig_km,orbit.op_ht_km,s_beam.emi_rcp,s_beam.beam_name,grp.grp_id,grp.freq_min,grp.freq_max,grp.bdwdth,grp.d_rcv,grp.d_inuse,grp.d_reg_limit,grp.d_prot_eff,grp.f_biu,emiss.seq_no,emiss.pwr_ds_max,emiss.design_emi,carrier_fr.freq_carr,channel.bandwidth,channel.freq_min,channel.freq_max,tpaconflicts
12,113540201,NaN,D,NaN,ICARUS,9.1/IA,04.07.2020,50,1.0,1.0,430.0,410.0,410.0,R,TAGU1,113662729,401.000,403.000,NaN,04.07.2013,NaN,04.07.2020,NaN,NaN,1.0,-74.8,1M50G1D--,402.0000,1500000.0,401.2500,402.7500,0:1:3:6:7:8:9:10:
23,115545003,NaN,D,NaN,TET-X,9.1/IA,03.02.2020,50,1.0,2.0,505.0,505.0,505.0,E,TXS1,120616227,2200.707,2206.707,NaN,03.02.2020,NaN,03.02.2027,NaN,NaN,1.0,-59.0,6M00G1DBW,2203.7070,6000000.0,2200.7070,2206.7070,11:
26,117545418,NaN,D,EUM,METOP-SG,9.1/IA,04.06.2020,50,1.0,4.0,830.0,830.0,830.0,R,SOR,117823868,2050.000,2059.000,NaN,04.06.2020,NaN,21.11.2024,NaN,NaN,1.0,-44.1,1M00G1D--,2056.0366,1000000.0,2055.5366,2056.5366,2:
44,117500020,NaN,D,NaN,ICARUS,11.2,10.01.2019,50,1.0,1.0,430.0,410.0,410.0,R,TAGU1,117616798,401.500,403.000,1500.0,10.01.2019,01.03.2020,04.07.2020,10.01.2019,C,1.0,-74.8,1M50G1D--,402.2500,1500000.0,401.5000,403.0000,0:1:3:6:7:8:9:10:
53,117500020,NaN,D,NaN,ICARUS,11.2,10.01.2019,50,1.0,1.0,430.0,410.0,410.0,R,TAGU1,117616798,401.500,403.000,1500.0,10.01.2019,01.03.2020,04.07.2020,10.01.2019,C,2.0,-75.0,1M10G1D--,402.2500,1100000.0,401.7000,402.8000,0:1:3:6:7:8:9:10:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771990,123545423,NaN,D,NaN,BRINDABELLA,9.1/IA,16.12.2023,50,119.0,24.0,380.0,380.0,380.0,E,STX,123763172,2232.250,2239.750,NaN,16.12.2023,NaN,16.12.2030,NaN,NaN,2.0,-60.4,2M00G7W--,2236.0000,2000000.0,2235.0000,2237.0000,4:
771994,123545423,NaN,D,NaN,BRINDABELLA,9.1/IA,16.12.2023,50,120.0,24.0,380.0,380.0,380.0,R,SRX,123763165,2052.250,2059.750,NaN,16.12.2023,NaN,16.12.2030,NaN,NaN,1.0,-48.8,7M50G7W--,2056.0000,7500000.0,2052.2500,2059.7500,2:
771996,123545423,NaN,D,NaN,BRINDABELLA,9.1/IA,16.12.2023,50,120.0,24.0,380.0,380.0,380.0,R,SRX,123763165,2052.250,2059.750,NaN,16.12.2023,NaN,16.12.2030,NaN,NaN,2.0,-43.0,2M00G7W--,2056.0000,2000000.0,2055.0000,2057.0000,2:
772004,123545423,NaN,D,NaN,BRINDABELLA,9.1/IA,16.12.2023,50,121.0,24.0,800.0,800.0,800.0,R,SRX,123763165,2052.250,2059.750,NaN,16.12.2023,NaN,16.12.2030,NaN,NaN,2.0,-43.0,2M00G7W--,2056.0000,2000000.0,2055.0000,2057.0000,2:


In [8]:
# save ddf_conflicts
outfile = './databases/germany_conflicts.csv'
ddf_conflicts.to_csv(outfile, index=False)

In [9]:
# get all the mission names
# sort by mission names
ddf_conflicts = ddf_conflicts.sort_values([' com_el.sat_name'])
names = ddf_conflicts[' com_el.sat_name'].unique()
print(names)

['AETHER-C' 'AETHER-K' 'BRINDABELLA' 'D-ISIPELE' 'DIAMANT' 'DSAT-E121.5'
 'DSAT-W026' 'DSAT-W129' 'GOSSAMER' 'ICARUS' 'IMPULSE-1' 'ISARLV1' 'LS-4'
 'METOP-SG' 'METOP-SG_EUM' 'NAMADGI' 'SIDLOC-BEECON' 'SUPA-NGSO-1'
 'TALOS-I' 'TET-X' 'UNICORN-2']
